# Initial Portfolio Allocation (Start of Jan 2025)

In [5]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Load data
btc = pd.read_excel("../data/pricing_data.xlsx", sheet_name='BTCUSD')
spy = pd.read_excel("../data/pricing_data.xlsx", sheet_name='SPY')
tlt = pd.read_excel("../data/pricing_data.xlsx", sheet_name='TLT - iShares 20+ Year Bond')

# Convert 'Date' column to datetime
btc['Date'] = pd.to_datetime(btc['Date'])
spy['Date'] = pd.to_datetime(spy['Date'])
tlt['Date'] = pd.to_datetime(tlt['Date'])

btc = btc[btc['Date'].dt.year == 2024]
spy = spy[spy['Date'].dt.year == 2024]
tlt = tlt[tlt['Date'].dt.year == 2024]

# Merge datasets on Date (no filtering for market hours)
data = pd.merge_asof(spy.sort_values('Date'), tlt.sort_values('Date'), on='Date', suffixes=('_SPY', '_TLT'))
data = pd.merge_asof(data.sort_values('Date'), btc.sort_values('Date'), on='Date')
data = data[['Date', 'Close_SPY', 'Close_TLT', 'Close']]  # Keep only Close prices

# Calculate log returns
data['Return_SPY'] = np.log(data['Close_SPY'] / data['Close_SPY'].shift(1))
data['Return_TLT'] = np.log(data['Close_TLT'] / data['Close_TLT'].shift(1))
data['Return_BTC'] = np.log(data['Close'] / data['Close'].shift(1))

# Add Cash column with zero returns
data['Return_Cash'] = 0

# Drop NaN rows after calculating returns
returns_df = data[['Return_SPY', 'Return_TLT', 'Return_BTC', 'Return_Cash']].dropna()

print(returns_df)

# Portfolio optimization function
def portfolio_sharpe(weights):
    weights = np.array(weights)
    mean_returns = returns_df.mean()
    cov_matrix = returns_df.cov()
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = portfolio_return / portfolio_volatility
    return -sharpe_ratio  # Negative for minimization

# Constraints: sum of weights = 1, cash between 1% and 5%
constraints = (
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Sum of weights = 1
    {'type': 'ineq', 'fun': lambda x: x[3] - 0.01},  # Cash >= 1%
    {'type': 'ineq', 'fun': lambda x: 0.05 - x[3]}   # Cash <= 5%
)
bounds = tuple((0, 1) for _ in range(4))  # No short selling (includes cash)

# Initial guess (equal weights)
initial_weights = [0.25, 0.25, 0.25, 0.25]

# Optimization
result = minimize(portfolio_sharpe, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights
optimal_weights = result.x

# Print optimal portfolio weights
print("Optimal Portfolio Weights:")
print(f"SPY: {optimal_weights[0]:.4f}")
print(f"TLT: {optimal_weights[1]:.4f}")
print(f"BTC: {optimal_weights[2]:.4f}")
print(f"Cash: {optimal_weights[3]:.4f}")

      Return_SPY  Return_TLT  Return_BTC  Return_Cash
1      -0.000816   -0.002946   -0.005420            0
2      -0.000170   -0.001109    0.003952            0
3       0.001301    0.001923   -0.000195            0
4       0.001938   -0.000254    0.003821            0
5      -0.000507   -0.001526   -0.003299            0
...          ...         ...         ...          ...
3255   -0.002041   -0.000911    0.002437            0
3256   -0.002453    0.000114   -0.003417            0
3257    0.002631    0.005112    0.014193            0
3258   -0.000339   -0.001474   -0.001216            0
3259   -0.001494   -0.000624   -0.001588            0

[3259 rows x 4 columns]
Optimal Portfolio Weights:
SPY: 0.8707
TLT: 0.0000
BTC: 0.1192
Cash: 0.0101
